In [1]:
#Empirical MSE for changing environment
#Uses new f_ave calculation weighting all times equally to new p(t) one.
from scipy.stats import t
from scipy.stats import norm;
import matplotlib.pyplot as plt
from PIL import Image
import random;
import numpy;
import scipy.integrate as integrate;
import scipy.special as special;
low = -1000;
high = 3000;

# Load Path Data

In [2]:
pathMeanTimes = [];
pathStdOfTimes = [];
pathDistances = [];
percentOfTimeInEachRegion = [];
P_I = [];
P_p = [];
regionConcent = [120,110,80,200,230,100,150];

#1801 Fry Street Falcon Heights, MN 55113 to 163 Bridge Street, Saint Paul, MN 55126 

#via Snelling Ave and Highway 96W. #T = 16min. D = 8.9 mi
#Follow Garden Ave to Snelling Ave
#1 min (0.2 mi)
#Turn left at the 2nd cross street onto Snelling Ave
#8 min (4.9 mi)
#Follow Hamline Ave N and Hwy 96 W to Bridge St in Shoreview
#7 min (3.8 mi)
pathMeanTimes.append(960);
pathStdOfTimes.append(60);
pathDistances.append(8.9);
P_I.append(0.2);
P_p.append(.25);
percentOfTimeInEachRegion.append([.33,.12,.3,0,0,.25,0]);

#via MN 36-E. T=15min. D=10.2 mi
#Get on MN-36 E in Roseville from Snelling Ave
#4 min (1.3 mi)
#Continue on MN-36 E. Take Exit 113 to Rice St. Take exit 45 from I-694 W/US-10 W
#8 min (6.7 mi)
#Follow Rice St to Bridge St
#4 min (2.2 mi)
pathMeanTimes.append(900);
pathStdOfTimes.append(45);
pathDistances.append(10.2);
P_I.append(0.3);
P_p.append(.20);
percentOfTimeInEachRegion.append([0,.18,0,.07,.12,.32,.31]);

#via Snelling Ave and County Rd F. T = 17min. D = 8.4 mi
#Follow Garden Ave to Snelling Ave
#1 min (0.2 mi)
#Turn left at the 2nd cross street onto Snelling Ave
#8 min (4.9 mi)
#Follow County Rd F West and Snail Lake Blvd to Bridge St in Shoreview
#8 min (3.3 mi)
pathMeanTimes.append(1020);
pathStdOfTimes.append(70);
pathDistances.append(8.4);
P_I.append(0.3);
P_p.append(.4);
percentOfTimeInEachRegion.append([.23,.22,.3,0,0,.25,0]);

#via Victorian T=19min. D = 8.6 mi
#Follow Garden Ave to Snelling Ave
#1 min (0.2 mi)
#Take Victoria St N to Snail Lake Rd in Shoreview
#16 min (7.6 mi)
#Turn right onto Snail Lake Rd
#2 min (0.6 mi)
#Turn left onto Rice St
#22 s (0.2 mi)
#Turn left onto Bridge St
#18 s (167 ft)
pathMeanTimes.append(1140);
pathStdOfTimes.append(75);
pathDistances.append(8.6);
P_I.append(0.1);
P_p.append(.05);
percentOfTimeInEachRegion.append([.07,.22,.17,.19,0,.35,0]);

#via MN 36 and Rice ST. T = 17.5 min D =8.9 mi
#Follow Garden Ave to Snelling Ave
#1 min (0.2 mi)
#Take MN-36 E and Rice St to Bridge St
#16 min (8.7 mi)
#Turn left onto Bridge St
#18 s (167 ft)
pathMeanTimes.append(1050);
pathStdOfTimes.append(65);
pathDistances.append(8.9);
P_I.append(0.1);
P_p.append(.1);
percentOfTimeInEachRegion.append([0,.18,0,.07,.11,.33,.31]);

print("Path Mean Times in seconds: ");
print(pathMeanTimes);
print("\nPath Standard Deviation of Times in seconds: ");
print(pathStdOfTimes);
print("\nPath Distances in miles:");
print(pathDistances);
print("\nPath Percent in each Region: ");
print(percentOfTimeInEachRegion);
print("\nP_I(x):");
print(P_I);
print("\nP_p(x):")
print(P_p);

Path Mean Times in seconds: 
[960, 900, 1020, 1140, 1050]

Path Standard Deviation of Times in seconds: 
[60, 45, 70, 75, 65]

Path Distances in miles:
[8.9, 10.2, 8.4, 8.6, 8.9]

Path Percent in each Region: 
[[0.33, 0.12, 0.3, 0, 0, 0.25, 0], [0, 0.18, 0, 0.07, 0.12, 0.32, 0.31], [0.23, 0.22, 0.3, 0, 0, 0.25, 0], [0.07, 0.22, 0.17, 0.19, 0, 0.35, 0], [0, 0.18, 0, 0.07, 0.11, 0.33, 0.31]]

P_I(x):
[0.2, 0.3, 0.3, 0.1, 0.1]

P_p(x):
[0.25, 0.2, 0.4, 0.05, 0.1]


In [3]:
#Alternate Individual Data

pathMeanTimes = [960, 900, 1020, 1140, 1050];

pathStdOfTimes = [60, 45, 70, 75, 65];

percentOfTimeInEachRegion = [[0.33, 0.12, 0.3, 0, 0, 0.25, 0], [0, 0.18, 0, 0.07, 0.12, 0.32, 0.31], [0.23, 0.22, 0.3, 0, 0, 0.25, 0], [0.07, 0.22, 0.17, 0.19, 0, 0.35, 0], [0, 0.18, 0, 0.07, 0.11, 0.33, 0.31]];

P_I = [0.2, 0.4, 0.1, 0.1, 0.2];





# Simulation

In [4]:
def training(numberOfPaths,pathAndTime,trainingDays,mInit):#minit = list of initialization for m
    m = mInit;
    c=[];
    tbarTotal=[];
    tbar = [];
    sumtijsquared = [];
    numberOfTimesRouteTaken = [];
    v=[];
    a = [];
    b = [];
    px = [];
    tloc = [];
    tscale = [];
    tdf = [];
    for path in range(0,numberOfPaths):
        c.append(0);
        v.append(1);
        a.append(0.01);
        b.append(0.01);
        c.append(0);
        tbarTotal.append(0);
        tbar.append(0);
        sumtijsquared.append(0);
        
        #numberOfTimesRouteTaken.append(0);
    for day in range(0,trainingDays):
        path = pathAndTime[day][0];
        c[path] = c[path]+1;
        sumtijsquared[path] = sumtijsquared[path] + pathAndTime[day][1]**2;
        tbarTotal[path] = tbarTotal[path] + pathAndTime[day][1];
    for path in range(0,numberOfPaths):
        if c[path]==0:
            tbar[path]=0;
        else:
            tbar[path] = tbarTotal[path]/c[path];
        

    for path in range(0,numberOfPaths):
        oldm = m[path];
        oldv = v[path];
        olda = a[path];
        oldb = b[path];
    
    
        v[path] = oldv/(1+c[path]*oldv);
        m[path] = (oldm/oldv+c[path]*tbar[path])*v[path];
        a[path] = olda+c[path]/2;
        b[path] = oldb + (1/2)*(pow(oldm,2)/oldv+sumtijsquared[path]-pow(m[path],2)/v[path]);
        tloc.append(m[path]);
        tscale.append(numpy.sqrt(b[path]*(1+v[path])/(a[path])));
        tdf.append(2*a[path]);
        px.append(c[path]/trainingDays);
        
    return [px,tdf,tloc,tscale,trainingDays];

In [5]:
def pt(t,pind,indPathMeans,indPathStds): #calculates true probability of time t for:
    #pind = list of individual true route selecction probabilities
    #indPathMeans = list of individual's mean travel times on each path
    #indPathStds = list of individual's standard deviations of travel times on each path
    total = 0;
    for i in range(0,len(pind)):
        total = total + norm.pdf(t,indPathMeans[i],indPathStds[i])*pind[i];
    return total

def probtBel(time,trainedParam):#thIs is updated belief p_B(time)
    result = 0.0;
    for x in range(0,len(trainedParam[0])):
        result = result + t.pdf(time, trainedParam[1][x], trainedParam[2][x], trainedParam[3][x])*trainedParam[0][x];
    return result

#print(str(probt(900)));

def probxgiventBel(x,time,trainedParam):#this is updted belief p_B(x|time)
    return t.pdf(time, trainedParam[1][x], trainedParam[2][x], trainedParam[3][x])*trainedParam[0][x]/probtBel(time,trainedParam)

In [6]:
#determining f(x,t) and average value


#low = 200
#high = 2650
#These are the limits of integraton for time in the average value calculations for f_ave and for the MSE.
#They must be carefully set so that you integrate over all significantly likely times, but not
#so wide that you get division by 0 for times for which p(t) is indistinguishable from 0.



def fexposure(path, time,concens):#path = path, time = time, concens = list of concentrations in each region
    #requires global specification of percent of time in each reagion, percentOfTimeInEachRegion
    #which is a list of lists.  There is one list for each path and that list is the list
    #of percents of times that path spends in each region.  This list must be of length len(concens)
    actualCITI = [];
    actualTimeInEachRegion = [];    
    for r in range(0,len(percentOfTimeInEachRegion[path])):
        actualTimeInEachRegion.append(percentOfTimeInEachRegion[path][r]*time);
    for regionNum in range(0,len(actualTimeInEachRegion)): #sum of atualCITI = f(x,t)
        actualCITI.append(concens[regionNum]*actualTimeInEachRegion[regionNum]);
    return sum(actualCITI);

def avef(baseconcentrations,pind,indPathMeans,indPathStds): #average f_environ(x,t) over all routes 
    #times and environments
    #baseconcentrations = list of base pollutant concentrations in each cell
    #pind = list of individual true route selecction probabilities
    #indPathMeans = list of individual's mean travel times on each path
    #indPathStds = list of individual's standard deviations of travel times on each path
    newvec = []
    for i in range (0,len(P_I)):
        pct_conc = sum(j[0] * j[1] for j in zip(percentOfTimeInEachRegion[i], baseconcentrations))
        newvec.append(pct_conc*pind[i]*pathMeanTimes[i]);
    return sum(newvec);



def avefold(baseconcentrations,pind,indPathMeans,indPathStds): #Old, but accurate
    #average f_environ(x,t) over all routes 
    #times and environments
    #baseconcentrations = list of base pollutant concentrations in each cell
    #pind = list of individual true route selecction probabilities
    #indPathMeans = list of individual's mean travel times on each path
    #indPathStds = list of individual's standard deviations of travel times on each path
    total = 0;
    #timeNormalization = integrate.quad(pt,low,high,args=(pind,indPathMeans,indPathStds))[0];
    timeNormalization = 1;
    for i in range(0,len(pind)):
        def f(t):
            #return fexposure(i,t,(baseconcentrations))*pt(t,pind,indPathMeans,indPathStds);
            return fexposure(i,t,(baseconcentrations))*norm.pdf(t,indPathMeans[i],indPathStds[i])*pind[i];
        total = total + integrate.quad(f,low,high,args=())[0];
    return total/timeNormalization;


In [7]:
#Generating path and time list

def pathAndTimeGen(days,pind,indPathMeans,indPathStds):
    indPathAndTime = [];
    #Pick a random path based on P_I = pind
    for i in range(0,days):
        randomNumber = random.uniform(0,1);
        pathLimit=0;
        for r in range(0,len(pind)):
            pathLimit = pathLimit+pind[r];
            if(randomNumber<pathLimit):
                randomPath = r;
                break;
        #Find a random time that it took for that path based on normal distribution for that path
        pathUserTime = random.normalvariate(indPathMeans[randomPath],indPathStds[randomPath]);
        indPathAndTime.append([randomPath,pathUserTime]);
    return indPathAndTime;

def environGen(regions,base,variation,days):#makes a list of environmental conditions
    #for days number of days
    #regions = number of Voronoi cells
    #base = base concentration list, a list of length = regions
    #variation = plus/minus range of variation in the values
    environment = [];
    for i in range(0,days):
        dailyConditions = [];
        for region in range(0,regions):
            dailyConditions.append(base[region]+random.uniform(-variation,variation));
        environment.append(dailyConditions);
    return [environment,base];


#userData = pathAndTimeGen(150);
#environment = environGen(7,[1.1,1.8,1.4,0.9,1.6,1.0,1.8],0.3,3);

#print(environment);

#print(environment[1]);


In [8]:
#########
#EMPERICAL MSE START


def gest(time,trainedParam, concens):#this is g(time)
    total = 0;
    for x in range(0,len(trainedParam[0])):
        total = total + fexposure(x,time,concens)*probxgiventBel(x,time,trainedParam)
    return total

#print("g(960) = "+str(gest(960)))


def empAveSquareError(trainedParam,pathAndTime, environment, startDay, periodLength):#Returns
    #empirically observed average square error
    #trainedParam = individual's hyperparameters after most current training
    #pathAndtime = list of [path,time] observations for the user on which we train and test
    #environment = list of lists, each of which is daily conditions in the Voronoi cells
    #startDay = first day on which to start testing.  Days start counting with 0 (!!!)
    #periodLength = length (in days) of this testing perios
    squaresum = 0;
    dayContribution = 0;
    for day in range(0,periodLength):
        path = pathAndTime[startDay+day][0];
        time = pathAndTime[startDay+day][1];
        concens = environment[startDay+day];
        dailyContribution = (fexposure(path,time,concens)-gest(time,trainedParam,concens))**2
        squaresum = squaresum + dailyContribution;
    return squaresum/periodLength;


In [9]:
#########
#THEORETICALLY MINIMAL MSE FOR CHANGING ENVIRONMENT BEGIN



#low = 100
#high = 2650
#These are the limits of integraton for time in the average value calculations for f_ave and for the MSE.
#They must be carefully set so that you integrate over all significantly likely times, but not
#so wide that you get division by 0 for times for which p(t) is indistinguishable from 0.

#########
#THEORETICALLY MINIMAL MSE START


def integrand(t, concens,pind,indPathMeans,indPathStds):#integrand in the MSE calculation
    #concens = list of pollutant concentrations in the cells
    #pind = list of individual true route selecction probabilities
    #indPathMeans = list of individual's mean travel times on each path
    #indPathStds = list of individual's standard deviations of travel times on each path
    term1 = 0;
    term2sum = 0;
    term2 = 0;
    for i in range(0,len(pathMeanTimes)):
        term1 = term1 + ((fexposure(i,t,concens))**2)*norm.pdf(t,indPathMeans[i],indPathStds[i])*pind[i];
    for i in range(0,len(pathMeanTimes)):
        term2sum = term2sum + fexposure(i,t,concens)*norm.pdf(t,indPathMeans[i],indPathStds[i])*pind[i];
    term2 = (term2sum**2)/pt(t,pind,indPathMeans,indPathStds);
    return term1 - term2;


def theoreticalmse(M,baseconc,confidencepct,pind,indPathMeans,indPathStds,variation):#computes confidence interval
    #for relative MSE with:
    #M = number of runs (trials)
    #baseconc = base concentration list
    #confidencepct  = confidence level in percent
    #pind = list of individual true route selecction probabilities
    #indPathMeans = list of individual's mean travel times on each path
    #indPathStds = list of individual's standard deviations of travel times on each path
    sqrunErAve = 0.0; #running average mean squared error
    frunAve =  0.0; # running average average value of f_epsilon(x,t)
    sqErtotal = 0.0; # total running square error
    relErrorList = [];
    errorList = [];
    averagefValue = avef(baseconc,pind,indPathMeans,indPathStds);
    for trial in range(0,M):
        concens = [0,0,0,0,0,0,0];
        for region in range(len(baseconc)):
            concens[region] = baseconc[region]+random.uniform(-variation,variation);#change environment
        mse = integrate.quad(integrand,low,high,args=(concens,pind,indPathMeans,indPathStds))[0];
        #integrate from low to high for about 6 standard deviations above and below mean times
        errorList.append(mse);
        relErrorList.append(mse/(averagefValue**2));
        #print(str(trial), end="");
    #
    std = numpy.std(relErrorList,ddof=1); #sample stdev of observed relative error
    tcv = t.ppf((100-confidencepct)/200,df = M-1,loc = 0, scale = 1); #critical value for our confidence
    mseest = numpy.mean(relErrorList);
    mseSqrtest = numpy.sqrt(mseest);
    confidence = [mseest - (-tcv*std)/numpy.sqrt(M), mseest +(-tcv*std)/numpy.sqrt(M)];
    #
    msePctest = mseest*100;
    mseSqrtPctest = mseSqrtest*100;
    msePctConfidence = [];
    mseSqrtPctConfidence = [];
    for i in range(0,len(confidence)):
        msePctConfidence.append(100*confidence[i]);
        mseSqrtPctConfidence.append(100*numpy.sqrt(confidence[i]));
    
    return [msePctest,msePctConfidence,mseSqrtPctest, mseSqrtPctConfidence,confidencepct];


#########
#THEORETICALLY MINIMAL MSE END



In [10]:
#Train-test regime for one person

def regime(pathAndTime,environment,indData,mseRuns,variation):#Train-test regime
    #pathAndTime = list of individual's (path,time) data
    #environment = [list of lists environmental conditions, base conditions]
        #that is, first entry in environment is a list of lists, each of which is a list of cell 
        #conditions and the second entry is the list of base cell conditions
    #indData = true probabilities and hyperparameter m initialization
        # [pind, indPathMeans, indPathStds, mInit] with
        #pind = list of individual true route selecction probabilities
        #indPathMeans = list of individual's mean travel times on each path
        #indPathStds = list of individual's standard deviations of travel times on each path
        #mInit = initialization of hyperparameter m
    #mseRuns = number of runs with which to estimate theoretical MSE (does not use pathAndTime)
    #variation = amount of variation on base conditions with respect to which to estimate theory MSE
    empMSEList = [];
    #TrainDuration = [7,14,22,32];#training duration in each cycle
    #TestDays = [7,8,10,12]#number of test days in each cycye
    #TrainDuration = [7,14,22,32,44,69];#training duration in each cycle
    #TestDays = [7,8,10,12,15,18]#number of test days in each cycye
    #TrainDuration = [20,30,45,65,90,120];#training duration in each cycle
    #TestDays = [10,15,20,25,30,35]#number of test days in each cycye
    #TrainDuration = [250,500,750];
    #TestDays = [250,250,250];
    TrainDuration = [5,10,20,30,40];
    TestDays = [250,250,250,250,250];
    numberOfPaths = len(indData[0]);
    #theoryMse = 5.28832505349;#DEBUG MSE TO SAVE TIME
    averagef  = avef(environment[1],indData[0],indData[1],indData[2]);
    thMseConf = theoreticalmse(mseRuns,environment[1],99.9,indData[0],indData[1],indData[2],variation);
    theoryMse = thMseConf[2];
    #theoryMse = theoreticalmse(mseRuns,environment[1],50,indData[0],indData[1],indData[2],variation)[2];
    for cycle in range(0,len(TrainDuration)):
        cycleAveSqErr = 0;
        trainedParam = [];
        #startDay = TrainDuration[cycle];
        startDay = (cycle+1)*250;
        periodLength = TestDays[cycle];
        trainedParam = training(numberOfPaths,pathAndTime,TrainDuration[cycle],indData[3]);
        cycleAveSqErr = empAveSquareError(trainedParam,pathAndTime,environment[0],startDay,periodLength);
        #print("\n"+"cycle sqrt of mse = "+str(numpy.sqrt(cycleAveSqErr)));
        empMSEList.append(100*numpy.sqrt(cycleAveSqErr)/(averagef));
        #empMSEList.append(round(numpy.sqrt(2),2));
    #print("\n");
    #print("Theory MSE pct = "+str(theoryMse));
    #print("Average f value ="+str(averagef));
    return [empMSEList,theoryMse,thMseConf];


        
    
    
    


In [11]:
#regimeVariation = 0.9;
#regimeBase = [1.1, 1.3, 0.9, 1.0, 1.2, 1.3, 0.9];
#regimeVariation = 0.4;
#regimeBase = [1.1,1.8,1.4,0.9,1.6,1.0,1.8];
#environment = environGen(7,regimeBase,regimeVariation,100);
#userTrueData = [P_I,pathMeanTimes,pathStdOfTimes,[0,0,0,0,0]];
#userPathData = [[1,554,],[2,311,],[1,760,],[2,409,],[2,478,],[1,679,],[2,378,],[2,329,],[1,723,],[2,523,],[3,420,],[2,482,],[2,423,],[1,708,],[4,773,],[2,396,],[1,617,],[2,361,],[2,367,],[2,369,],[2,433,],[2,366,],[2,382,],[1,691,],[1,728,],[2,373,],[2,522,],[2,369,],[1,744,],[2,434,],[2,378,],[2,368,],[2,358,],[1,663,],[2,372,],[2,369,],[2,362,],[2,350,],[1,669,],[1,700,],[2,383,],[2,367,],[2,381,],[2,383,],[2,368,],[1,725,],[1,626,],[1,764,]]
#
#
#TestMSE = regime(userPathData,environment,userTrueData,20,regimeVariation);

##print(str(TestMSE));

#for i in range(0,len(TestMSE[0])):
#    print(str(TestMSE[0][i])+"  ", end="");
#    #if i == len(MSElist)-1:
##    print(" ")
#print("\n"+str(TestMSE[1]));

In [12]:

def multipleRegime(noPeople,peopleBase,peopleVar,regimeBaseEnv,regimeVar,dataDays,theoryRuns):#
    #noPeople = number of people doing the regime
    #dataDays = number of days for which we have/want data
    empMSElist = [];
    theoryMSElist = [];
    theoryMSEConflist = [];
    environment = environGen(7,regimeBaseEnv,regimeVar,dataDays);#here for all people
     #having the same environment
    for people in range(0,noPeople):
    #print("\n");
        indPathMeans = [];
        indPathStds = [];
        pind = [];
        pindTemp = [];
        denominator = 0;
        for path in range(0,len(peopleBase[0])):
            pindTemp.append(peopleBase[0][path]+random.uniform(0,peopleVar[0]))
            indPathMeans.append(peopleBase[1][path]+random.uniform(-peopleVar[1],peopleVar[1]));
            indPathStds.append(peopleBase[2][path]+random.uniform(-peopleVar[2],peopleVar[2]));
        for path in range(0,len(peopleBase[0])):
            denominator = sum(pindTemp);
            pind.append(pindTemp[path]/denominator);
        #print(str(pind));
        #indPathStds = peopleBase[1]+random.uniform(-peopleVar[1],peopleVar[1]);
        userPathData = pathAndTimeGen(dataDays,pind,indPathMeans,indPathStds);
        #environment = environGen(7,regimeBaseEnv,regimeVar,dataDays);#here for each person
        #having a different environment
        personTrueData = [pind,indPathMeans,indPathStds,peopleBase[3]];
        personMSE = regime(userPathData,environment,personTrueData,theoryRuns,regimeVar);
        empMSElist.append(personMSE[0]);
        theoryMSElist.append(personMSE[1]);
        theoryMSEConflist.append(personMSE[2]);
        
    return [empMSElist,theoryMSElist,theoryMSEConflist];

#peopleBase = [P_I, pathMeanTimes, pathStdOfTimes];
#peopleVar = [0.5,400,50];

#regimeCycle = multipleRegime(10, peopleBase, peopleVar, regimeBase,regimeVariation,6000,40)

#print(str(regimeCycle));
    #print(str(MSElist));
#    for i in range(0,len(MSElist)):
#        print(str(MSElist[i])+"  ", end="");
#        if i == len(MSElist)-1:
#            print(" ")

    
            

In [13]:
peopleBase = [P_I, pathMeanTimes, pathStdOfTimes,[0,0,0,0,0]];
peopleVar = [0.5,400,40];
regimeVariation1 = 1;
regimeBase1 = [4,3,2,5,5,2,6];
#environment = environGen(7,regimeBase1,regimeVariation1,100);
#userTrueData = [P_I,pathMeanTimes,pathStdOfTimes,[0,0,0,0,0]];


#regimeCycle1 = multipleRegime(100, peopleBase, peopleVar, regimeBase1,regimeVariation1,6000,100)
#print(regimeCycle1);


In [14]:
def sumarize(regimeResults):
    rounds = [];
    averages = [numpy.mean(regimeResults[1])];
    high = [numpy.max(regimeResults[1])];
    low = [numpy.min(regimeResults[1])];
    for i in range(0,len(regimeResults[0][0])):
        rounds.append([]);
    for person in range(0,len(regimeResults[1])):
        for i in range(0,len(rounds)):
            rounds[i].append(regimeResults[0][person][i]);
    for i in range(0,len(rounds)):
        averages.append(numpy.mean(rounds[i]));
        low.append(numpy.min(rounds[i]));
        high.append(numpy.max(rounds[i]));
    return [averages, low, high];


#summary = sumarize(regimeCycle1);

#print(regimeCycle1[1]);
#print("\n");
#for i in range(0,len(regimeCycle1[0])):
#    print(regimeCycle1[0][i]);

#print("\n\n");
#print(summary[0]);
#print(summary[1]);
#print(summary[2]);

In [15]:
peopleBase = [P_I, pathMeanTimes, pathStdOfTimes,[0,0,0,0,0]];
peopleVar = [0.5,400,40];
regimeVariation2 = 0.75;
regimeBase2 = [3,2,3,4,3,4,4];
#environment = environGen(7,regimeBase,regimeVariation,100);
#userTrueData = [P_I,pathMeanTimes,pathStdOfTimes,[0,0,0,0,0]];


#regimeCycle2 = multipleRegime(100, peopleBase, peopleVar, regimeBase2,regimeVariation2,6000,100)
#print(regimeCycle2);

[[[8.3938805541238857, 8.2211850215955273, 5.4892647400677212, 4.6477646590803321, 4.5581966418526223], [7.5574435793294459, 5.7710448771898735, 6.070083184391903, 5.4688983242777542, 5.3192158165169792], [7.6226752072661261, 8.1617807622046818, 5.801575202135079, 5.7363581127412751, 4.8709719830337743], [6.7689206946434295, 7.1040698010844761, 5.4733517036584356, 5.8887635393768765, 5.309821439940734], [4.9988733554718001, 3.781085524752688, 3.6653790908956698, 3.2922403925277717, 2.7740671174176348], [8.2306766188658411, 7.5212155531233611, 6.1036789170889065, 7.5361769101281046, 7.2580843357458384], [10.627804798917126, 8.0111446241942836, 4.5574873584552025, 4.5340232473739759, 3.9158575261215773], [10.442031192466491, 8.4674046260388547, 6.8216595778077256, 5.5103282293207227, 5.5389571876823593], [9.948881634573663, 6.7216507590647563, 6.5207452188357422, 6.5995727399577513, 6.5669433274575022], [5.0659853189581083, 5.8177181606309318, 3.3826415697697336, 4.1365906725619075, 3.83

In [16]:
peopleBase = [P_I, pathMeanTimes, pathStdOfTimes,[0,0,0,0,0]];
peopleVar = [0.5,400,40];
regimeVariation3 = 0.5;
regimeBase3 = [2,2,3,2,3,2,3];
#environment = environGen(7,regimeBase,regimeVariation,100);
#userTrueData = [P_I,pathMeanTimes,pathStdOfTimes,[0,0,0,0,0]];


#regimeCycle3 = multipleRegime(100, peopleBase, peopleVar, regimeBase3,regimeVariation3,6000,100)
#print(regimeCycle3);

[[[5.0232056896532908, 4.5473240856993007, 3.5353485034437866, 2.8640071483420853, 2.684227602159361], [7.2303489779111763, 9.0282223543527387, 2.6466382765218848, 2.0263369424319353, 1.8650205010659322], [5.3490413346314671, 5.0462043902875733, 4.3958153992652873, 4.3602874443457882, 4.3046585668372375], [6.1076535129386134, 4.415907320433841, 4.2625370780831027, 2.8195211147766717, 3.0909505705358007], [4.7442209974074681, 3.8187921827246445, 2.9457665836693363, 2.5703072487937733, 2.7431796673805873], [6.3652494997131175, 5.2500463720733199, 3.8561048190305813, 3.8223800684093634, 4.1908377171182458], [5.9986222503920201, 5.0729397735584802, 3.0876057527076708, 3.9247120352045641, 3.5057497427430011], [3.8465962034366674, 4.0867584315236414, 5.1847836637491227, 4.6332893861716373, 3.9569854862185529], [5.3225970784725591, 4.4151393127076402, 4.1868935128731275, 4.0080880774248486, 3.4542384695184194], [5.8032104606362829, 7.4044232303063842, 4.2936367658534662, 4.2491697159483595, 3

In [17]:
#def sumarize(regimeResults):
#    rounds = [];
#    averages = [numpy.mean(regimeResults[1])];
#    high = [numpy.max(regimeResults[1])];
#    low = [numpy.min(regimeResults[1])];
#    for i in range(0,len(regimeResults[0][0])):
#        rounds.append([]);
#    for person in range(0,len(regimeResults[1])):
#        for i in range(0,len(rounds)):
#            rounds[i].append(regimeResults[0][person][i]);
#    for i in range(0,len(rounds)):
#        averages.append(numpy.mean(rounds[i]));
#        low.append(numpy.min(rounds[i]));
#        high.append(numpy.max(rounds[i]));
#    return [averages, low, high];


#summary = sumarize(regimeCycle1);

#print(regimeCycle1[1]);
#print("\n");
#for i in range(0,len(regimeCycle1[0])):
#    print(regimeCycle1[0][i]);

#print("\n\n");
#print(summary[0]);
#print(summary[1]);
#print(summary[2]);

#summary = sumarize(regimeCycle1);

#print("Regime Cycle 1 TheoryMSEs = "+str(regimeCycle1[1])+"\n");
#print("Regime Cycle 2 TheoryMSEs = "+str(regimeCycle2[1])+"\n");
#print("Regime Cycle 3 TheoryMSEs = "+str(regimeCycle3[1])+"\n");


#print(str(numpy.mean(regimeCycle1[1])));
#print("\n");
#for i in range(len(regimeCycle1[0])):
#    print(str(regimeCycle1[0][i]));
#print("\n");
#print(str(summary));


In [19]:
for k in range(0,5):
    print(regimeCycle3[0][15*k],regimeCycle3[1][15*k]);

[5.0232056896532908, 4.5473240856993007, 3.5353485034437866, 2.8640071483420853, 2.684227602159361] 2.45119708895
[4.6227609263309004, 3.6746462358682597, 5.3395745396387344, 4.2266115674059996, 3.8283330902317689] 3.7063671002
[5.2196423263820257, 4.1018267972415607, 3.481353010135487, 3.435073308220522, 3.3795996423362817] 2.994473851
[4.1914420808094404, 3.0902863106467944, 3.6304296068394106, 3.8377295853222977, 3.842392308759242] 2.60890804226
[5.5288660923629251, 4.3491959155629614, 4.3719450863020679, 3.4049248693574357, 3.2906029586389631] 3.05729167804


In [21]:
def sumarize(regimeResults):
    rounds = [];
    averages = [numpy.mean(regimeResults[1])];
    high = [numpy.max(regimeResults[1])];
    low = [numpy.min(regimeResults[1])];
    for i in range(0,len(regimeResults[0][0])):
        rounds.append([]);
    for person in range(0,len(regimeResults[1])):
        for i in range(0,len(rounds)):
            rounds[i].append(regimeResults[0][person][i]);
    for i in range(0,len(rounds)):
        averages.append(numpy.mean(rounds[i]));
        low.append(numpy.min(rounds[i]));
        high.append(numpy.max(rounds[i]));
    return [averages, low, high];


summary = sumarize(regimeCycle3);

#print(regimeCycle1[1]);
#print("\n");
#for i in range(0,len(regimeCycle1[0])):
#    print(regimeCycle1[0][i]);

print("\n\n");
print(summary[0]);
print(summary[1]);
print(summary[2]);




[3.4044614984502237, 5.484396046025025, 4.6745399099930998, 4.0505875661182378, 3.9481090771029375, 3.6453912056897684]
[0.99350065008214694, 2.707562640836497, 2.0468842200278972, 1.2859237083494168, 1.2947777354268832, 1.0696982598676281]
[5.0569297390969421, 8.840231851907669, 9.0282223543527387, 6.8552371130757921, 6.541318352292846, 5.9760000274679408]


In [27]:
#First 10 MSE 99.9% confidence intervals


print("Medium");
for k in range(0,10):
    print(regimeCycle2[2][k][3]);
print(" ");
print("Low");
for k in range(0,10):
    print(regimeCycle3[2][k][3]);
print(" ");

Medium
[4.0484342786245167, 5.3879071523917652]
[3.7659742767174471, 5.242491758544678]
[4.1484006574145784, 5.446354577080136]
[4.2977114977722159, 5.7652010227257362]
[2.331355091240586, 3.2880066693856258]
[5.2965663827096217, 6.8631559192341598]
[2.8037660024564244, 4.030397484014852]
[3.9038447943092649, 5.3232331319353667]
[5.2481676991142505, 7.1098025079269238]
[2.9369359810900537, 3.996129750851904]
 
Low
[1.922839757395753, 2.8843407574597504]
[1.6196251014531753, 1.9754492532182768]
[3.3533523401736578, 4.2978121736115416]
[2.7322917398896132, 3.4172713306080875]
[2.2627173078578955, 3.3210854083482158]
[3.3648352184113302, 4.0925636472781726]
[2.7872557193240151, 3.5616933656883814]
[2.9014529180032724, 3.7689879112009836]
[2.9049396650693327, 4.0169433625474831]
[3.2515827927052556, 4.3179351602246756]
 


In [30]:
#Average 99.9% confidence interval width

print("Medium");
medium = 0;
for k in range(0,100):
    medium = medium + (0.01)*(regimeCycle2[2][k][3][1]-regimeCycle2[2][k][3][0])
print(medium);
print(" ");
print("Low");
low = 0;
for k in range(0,100):
    low = low + (0.01)*(regimeCycle3[2][k][3][1]-regimeCycle3[2][k][3][0])
print(low);


Medium
1.18094081304
 
Low
0.948707210006
